In [8]:
from keras.layers import Input, Conv2D, MaxPooling2D, concatenate
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
import numpy as np
from PIL import Image

In [9]:
target_size = (900, 400)

In [10]:
# Function to load and preprocess an image
def load_and_preprocess_image(image_path):
    img = Image.open(image_path)
    img = img.convert('RGB')
    img = img.resize(target_size[::-1], Image.Resampling.LANCZOS)
    # img = image.load_img(image_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalize pixel values to between 0 and 1
    return img_array

In [11]:
input_person_image = load_and_preprocess_image('person_image.png')  # Replace with the actual path
input_dress_image = load_and_preprocess_image('dress_image.png')  # Replace with the actual path

In [26]:
def create_network():
    # Load a pre-trained VGG16 model for feature extraction
    vgg = VGG16(weights='imagenet', include_top=False)

    # Define two input layers
    person_input = Input(shape=(target_size[0], target_size[1], 3))
    clothing_input = Input(shape=(target_size[0], target_size[1], 3))

    # Extract features using VGG16 for both inputs
    person_features = vgg(person_input)
    print(person_input.shape)
    print(person_features.shape)
    print(person_features)
    clothing_features = vgg(clothing_input)

    # Here, you would need a custom layer to combine these features
    # This is a non-trivial task and would involve spatial transformations
    # and possibly training a separate model to learn these transformations
    combined_features = concatenate([person_features, clothing_features])

    # Add some convolutional layers
    x = Conv2D(64, (3, 3), activation='relu', padding='same', strides=(1, 1))(combined_features)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', strides=(1, 1))(x)

    # Output layer: output the same size as the input image
    output = Conv2D(3, (3, 3), activation='sigmoid')(x)
    print(output.shape)

    # Create the model
    model = Model(inputs=[person_input, clothing_input], outputs=output)
    return model


In [27]:
# Create the model
model = create_network()
model.compile(optimizer='adam', loss='mean_squared_error')

(None, 900, 400, 3)
(None, 28, 12, 512)
KerasTensor(type_spec=TensorSpec(shape=(None, 28, 12, 512), dtype=tf.float32, name=None), name='vgg16/block5_pool/MaxPool:0', description="created by layer 'vgg16'")
(None, 12, 4, 3)


In [25]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_14 (InputLayer)       [(None, 900, 400, 3)]        0         []                            
                                                                                                  
 input_15 (InputLayer)       [(None, 900, 400, 3)]        0         []                            
                                                                                                  
 vgg16 (Functional)          (None, None, None, 512)      1471468   ['input_14[0][0]',            
                                                          8          'input_15[0][0]']            
                                                                                                  
 concatenate_4 (Concatenate  (None, 28, 12, 1024)         0         ['vgg16[0][0]',         

In [14]:
merged_image_array = model.predict([input_person_image, input_dress_image])

1/1 [==============================] - 1s 831ms/step


In [15]:
# Save the merged image as a PNG file
merged_image_path = 'merged_image.png'
image.save_img(merged_image_path, merged_image_array[0])